In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

1.1 Implement gradient based factorisation 

In [28]:
from typing import Tuple
def sgd_factorise(A: torch.Tensor, rank: int, num_epochs = 1000, lr = 0.01) -> Tuple[torch.Tensor, torch.Tensor]:
  m, n = A.shape
  U = torch.rand(m, rank)
  V = torch.rand(n, rank)

  for epoch in range(num_epochs):
    for r in range(m):
      for c in range(n):
        e = A[r, c] - U[r] @ V[c].T
        U[r] = U[r] + lr * e * V[c]
        V[c] = V[c] + lr * e * U[r]

  return U, V

2.1 Factorise and compute reconstruction error

In [29]:
A = torch.tensor(
    [[0.3374, 0.6005, 0.1735],
     [3.3359, 0.0492, 1.8374],
     [2.9407, 0.5301, 2.2620]])

U, V = sgd_factorise(A, rank = 2)

print("A = " ,A) 
print("U = " ,U)
print("V = " ,V)
print()

A_sgd = U @ V.T
sgd_loss = torch.nn.functional.mse_loss(input = A_sgd , target = A, reduction = 'sum')
print(sgd_loss)

A =  tensor([[0.3374, 0.6005, 0.1735],
        [3.3359, 0.0492, 1.8374],
        [2.9407, 0.5301, 2.2620]])
U =  tensor([[ 0.6087, -0.1197],
        [ 0.4773,  1.6923],
        [ 1.1350,  1.2965]])
V =  tensor([[ 0.7014,  1.7256],
        [ 0.7938, -0.2314],
        [ 0.7831,  0.9428]])

tensor(0.1221)


3.1 Compare  to the truncated SVD

Calculating reconstruction error using SVD

In [30]:
U, S, V = torch.svd(A)

print(S.shape)
S[2] = 0
print(S)

A_svd = U @ torch.diag(S) @ V.T

print(A_svd)

svd_loss = torch.nn.functional.mse_loss(A, A_svd, reduction='sum')
print(svd_loss)

torch.Size([3])
tensor([5.3339, 0.6959, 0.0000])
tensor([[ 0.2245,  0.5212,  0.3592],
        [ 3.2530, -0.0090,  1.9737],
        [ 3.0378,  0.5983,  2.1023]])
tensor(0.1219)


Compare the difference between the algorithm and SVD

Eckart-Young-Mirsky Theorem: 
https://rich-d-wilkinson.github.io/MATH3030/3-5-lowrank.html

In [31]:
diff = torch.norm(sgd_loss - svd_loss)
print(diff)

tensor(0.0002)


3.1 Implement Masked Factorisation

In [32]:
from typing import Tuple
def sgd_factorise_masked(A: torch.Tensor, M: torch.Tensor, rank: int, 
                         num_epochs = 1000, lr = 0.01) -> Tuple[torch.Tensor, torch.Tensor]:
  m, n = A.shape
  U = torch.rand(m, rank)
  V = torch.rand(n, rank)

  for epoch in range(num_epochs):
    for r in range(m):
      for c in range(n):
        if M[r, c] == 1:
          e = A[r, c] - U[r] @ V[c].T
          UOrigin = U[r]
          U[r] = U[r] + lr * e * V[c]
          V[c] = V[c] + lr * e * UOrigin

  return U, V

3.2 Reconstruct a matrix

In [33]:
AWithNAN = torch.Tensor(
    [[0.3374, 0.6005, 0.1735],
     [torch.nan, 0.0492, 1.8374],
     [2.9407, torch.nan, 2.2620]]
)

M = torch.Tensor(
    [[1,1,1],
     [0,1,1],
     [1,0,1]]
)

U, V = sgd_factorise_masked(A, M, rank = 2) 

A_original = U @ V.T
print(A_original)
loss = torch.nn.functional.mse_loss(A_original, A, reduction='sum')
print(loss)

tensor([[0.3439, 0.5989, 0.1657],
        [2.2581, 0.0494, 1.8358],
        [2.9393, 0.7159, 2.2642]])
tensor(1.1962)
